In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpynn as nn

# Example 3

### Convolutional neural network

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/competitions/digit-recognizer/data and place it into the *data* directory.

In [ ]:
data = pd.read_csv('data/mnist/train.csv')
data.head()

In [ ]:
data_enc = nn.preprocessing.pd_one_hot_encode(data, columns=['label'])
data_enc.head()

In [ ]:
tensor = nn.tensor.pd_to_tensor(data_enc)
train, val, test = nn.preprocessing.split_train_val_test(tensor, ratio_val=0.01, ratio_test=0.1)

In [ ]:
x_train, y_train = nn.preprocessing.split_features_labels(train, 784)
x_val, y_val = nn.preprocessing.split_features_labels(val, 784)
x_test, y_test = nn.preprocessing.split_features_labels(test, 784)

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 1 , 28, -1))
x_val = x_val.reshape((x_val.shape[0], 1, 28, -1))
x_test = x_test.reshape((x_test.shape[0], 1, 28, -1))

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')

print (f'{x_val.shape=}')
print (f'{y_val.shape=}')

print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

Normalization

In [ ]:
x_train = x_train / 255
x_val = x_val / 255
x_test = x_test / 255

### Step 2: Build the neural network structure

In [ ]:
model = nn.networks.Sequential(input_shape=(1, 28, 28), mdl_layers=[
    nn.layers.Convolution(out_channels=8, kernel_shape=(3, 3), act_fn=nn.activations.Relu(), norm_fn=nn.norms.Layernorm()),
    nn.layers.MaxPooling(p_window=(2, 2)),
    nn.layers.Flatten(),
    nn.layers.Linear(out_channels=64, act_fn=nn.activations.Relu(), norm_fn=nn.norms.Layernorm()),
    nn.layers.Linear(out_channels=10, act_fn=nn.activations.Softmax())
])

The network is compiled to internally connect it's layers and initialize the model.

In [ ]:
model.compile(
    optimizer=nn.optimizers.Adam(l_r=1e-3),
    loss_fn=nn.losses.Crossentropy(),
    metric=nn.metrics.accuracy
)

In [ ]:
model.summary()

### Step 3: Train the model

In [ ]:
hist = model.train(x_train, y_train, epochs=20, batch_size=512, val_data=(x_val, y_val))

In [ ]:
model.plot_training_loss(hist)

### step 4: Evaluate the model

In [ ]:
model.evaluate(x_test, y_test)

### Explore the inner workings
Pick a random image from the testing dataset.

In [ ]:
i = random.randint(0, x_test.len)
image = np.moveaxis(x_test[i].data, 0, -1)
plot = plt.imshow(image, cmap='gray')

Use it to predict a number and show the probability distribution of the outcome.

In [ ]:
image_tensor = nn.expand_dims(x_test[i], 0)
p, _ = model(image_tensor)
plot = plt.bar(np.arange(0, 10), p.reshape((10,)).data)
plt.xlabel('label')
plt.ylabel('probability')

Every layer of the model can be accessed to explore their output. Here we iterate over all the kernels of the convolutional layer to explore what they learned to focus on in images.

In [ ]:
model.plot_conv_channels()